In [2]:
import numpy as np
import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F

from competence_estimation.utils import load_data, get_network_weights, mix_open
from competence_estimation.scores  import create_score_function
from competence_estimation.metrics  import compute_metric

import pandas as pd
import matplotlib.pyplot as plt

# Short Introduction

The following files are required:
* {x}_id_train.npy
* {x}_id_val.npy
* {x}_id_test.npy
where x = features, logits, labels

and the same for ood samples:
* {x}_ood
where x = features, logits, labels

Additionally we require the weights w and biases b of the last layer

# Data and Config Loading

In [3]:
with open("config.yml", "r") as file:
    config = yaml.safe_load(file)

# Create Plot

In [ ]:
# Here you have to include your own path
# dataset_path = "/home/DATA/ITWM/lorenzp/"
dataset_path = "/home/lorenzp/workspace/competence_estimation/features/cifar10"
# algorithm = "ERM"
model = "resnet18"

load_data = model
mode ="spatial"

w = np.load(f"{dataset_path}/spatial/{model}_W.npy")
b = np.load(f"{dataset_path}/spatial/{model}_b.npy")

# Features/logits/labels Trainings data
features_id_train =  np.load(f"{dataset_path}/spatial/features_{mode}_{model}_train.npy")[:30000]
logits_id_train = np.load(f"{dataset_path}/spatial/logits_{mode}_{model}_train.npy")[:30000]
labels_id_train =  np.load(f"{dataset_path}/spatial/labels_{mode}_{model}_train.npy")[:30000]

# Features/logits/labels Validation data
features_id_val = np.load(f"{dataset_path}/spatial/features_{mode}_{model}_train.npy")[30000:]
logits_id_val = np.load(f"{dataset_path}/spatial/logits_{mode}_{model}_train.npy")[30000:]
labels_id_val = np.load(f"{dataset_path}/spatial/labels_{mode}_{model}_train.npy")[30000:]

# Features/logits/labels Test data
features_id_test = np.load(f"{dataset_path}/spatial/features_{mode}_{model}_test.npy")[:5000]
logits_id_test = np.load(f"{dataset_path}/spatial/logits_{mode}_{model}_test.npy")[:5000]
labels_id_test = np.load(f"{dataset_path}/spatial/labels_{mode}_{model}_test.npy")[:5000]


print(
    features_id_train.shape,
    features_id_val.shape,
    features_id_test.shape
)

metrics = ['quantiles']
results = {}
results_frac = {}
results_id_test = {}
results_id_test_frac = {}
attacks = ['nor', 'fgsm', 'linfpgd', 'aa', 'linfdf', 'l2df']
# attacks = ['nor', 'fgsm']
score_fct = ["max_softmax", "max_logit", "vim", "mahalanobis", "knn", "energy", "energy_react", "GMM",  "PCA"]
# score_fct = ["max_softmax", "max_logit", "vim"]
# attacks = ['pgd']
# score_fct = [ "vim" ]


for it, attack in enumerate(attacks):
    print("attack: ", attack)
    results[attack] = {}
    results_frac[attack] = {}
    results_id_test[attack] = {}
    results_id_test_frac[attack] = {}
    
    if it == 0:
        features_ood = np.load(f"{dataset_path}/features_{mode}_{model}_test.npy")[5000:]
        logits_ood = np.load(f"{dataset_path}/logits_{mode}_{model}_test.npy")[5000:]
        labels_ood = np.load(f"{dataset_path}/labels_{mode}_{model}_test.npy")[5000:]
    else:
        features_ood = np.squeeze(np.load(f"{dataset_path}/features_{attack}_{mode}_{model}_test.npy"))[5000:]
        logits_ood = np.load(f"{dataset_path}/logits_{attack}_{mode}_{model}_test.npy")[5000:]
        labels_ood = np.load(f"{dataset_path}/labels_{attack}_{mode}_{model}_test.npy")[5000:]
    
    for it2, score_function_name in enumerate(score_fct):
        scores_id_val, score_function = create_score_function(
            features_id_train,
            logits_id_train,
            labels_id_train,
            features_id_val,
            logits_id_val,
            labels_id_val,
            w,
            b,
            score_function =  score_function_name,
            **config
        )
        scores_id_train = score_function(features_id_train, logits_id_train)
        scores_id_test  = score_function(features_id_test, logits_id_test)
        scores_ood_test = score_function(features_ood, logits_ood)
        
        print(features_id_train.shape)
                                         
        outs = compute_metric(
            scores_id_val,
            scores_id_test,
            scores_ood_test,
            features_id_train,
            logits_id_train,
            labels_id_train,
            features_id_test,
            logits_id_test,
            labels_id_test,
            features_ood,
            logits_ood,
            labels_ood,
            metrics=metrics
        )

        print(score_function_name, outs)
        results[attack][score_function_name] = outs['n_95']
        results_frac[attack][score_function_name] = outs['n_95_frac']
        results_id_test[attack][score_function_name] = outs['n_95_id_test']
        results_id_test_frac[attack][score_function_name] = outs['n_95_frac_id_test']
        
keys = list(results.keys())
methods  = list(results[keys[0]].keys())

table = {}
table_frac = {}

for method in methods: 
    table[method] = []
    table_frac[method] = []
    
    for  key in keys:
        table[method].append(results[key][method])
        table_frac[method].append(results_frac[key][method])

(30000, 512) (20000, 512) (5000, 512)
attack:  nor
(30000, 512)
max_softmax {'n_95_frac': 0.813, 'n_95': 0.9938499331474304, 'n_95_frac_id_test': 0.8086, 'n_95_id_test': 0.9930744767189026}
(30000, 512)
max_logit {'n_95_frac': 0.8382, 'n_95': 0.9868766665458679, 'n_95_frac_id_test': 0.8368, 'n_95_id_test': 0.9887667298316956}
(30000, 512)
vim {'n_95_frac': 0.8538, 'n_95': 0.9845396876335144, 'n_95_frac_id_test': 0.853, 'n_95_id_test': 0.9819460511207581}
(30000, 512)
mahalanobis {'n_95_frac': 0.8758, 'n_95': 0.9783055782318115, 'n_95_frac_id_test': 0.8794, 'n_95_id_test': 0.9747555255889893}
(30000, 512)
knn {'n_95_frac': 0.8386, 'n_95': 0.9883138537406921, 'n_95_frac_id_test': 0.8398, 'n_95_id_test': 0.9880924224853516}
(30000, 512)
energy {'n_95_frac': 0.8396, 'n_95': 0.9864221215248108, 'n_95_frac_id_test': 0.8392, 'n_95_id_test': 0.9887988567352295}
(30000, 512)
energy_react {'n_95_frac': 0.8358, 'n_95': 0.9863603711128235, 'n_95_frac_id_test': 0.8404, 'n_95_id_test': 0.98810088634

In [14]:
results_frac

{'nor': {}}

In [ ]:
results_id_test_frac

In [ ]:
keys

In [ ]:
table.keys()

In [ ]:
results['nor']

In [ ]:
results['fgsm']

In [ ]:
color=["#ff7f0e", "#2ca02c", '#d62728', '#9467bd', "#8c564b", "#e377c2", '#7f7f7f', '#bcbd22', '#17becf'][:len(table)]

raw_data = {
     'plan_type':  ['Benign', 'FGSM-$L^\infty$', 'PGD-$L^\infty$', 'AA' 'DF-$L^\infty$', 'DF-$L^2$'][:len(keys)],
}

for method in methods:
    raw_data[ method] = table[method]

df = pd.DataFrame(raw_data, columns=methods)
ax = df.plot.bar(rot=0, color=color[:len(table)], width=0.9)

for p in ax.patches[0:]:
    h = p.get_height()
    x = p.get_x()+p.get_width()/2.
    if h != 0:
        ax.annotate("%g" % p.get_height(), xy=(x, h), xytext=(0, 3), rotation=90, 
                   textcoords="offset points", ha="center", va="bottom")

plt.title("Spatial Data trained on ResNet18")
      
ax.set_ylim(0, 1.2)
ax.legend(title="Detectors", bbox_to_anchor=(1.0, 1), loc='upper left')
ax.set_xticklabels(raw_data["plan_type"])
plt.xlabel("attacks")
plt.ylabel("n_95")

plt.savefig(f"plots/{mode}_{model}_95.png", bbox_inches='tight', dpi=300)

In [ ]:
color=["#ff7f0e", "#2ca02c", '#d62728', '#9467bd', "#8c564b", "#e377c2", '#7f7f7f', '#bcbd22', '#17becf'][:len(table)]

raw_data = {
     'plan_type':  ['Benign', 'FGSM-$L^\infty$', 'PGD-$L^\infty$', 'DF-$L^\infty$', 'DF-$L^2$'][:len(keys)],
}

for method in methods:
    raw_data[method] = table_frac[method]

df = pd.DataFrame(raw_data, columns=methods)
ax = df.plot.bar(rot=0, color=color[:len(table)], width=0.9)

for p in ax.patches[0:]:
    h = p.get_height()
    x = p.get_x()+p.get_width()/2.
    if h != 0:
        ax.annotate("%g" % p.get_height(), xy=(x, h), xytext=(0, 3), rotation=90, 
                   textcoords="offset points", ha="center", va="bottom")

plt.title("Spatial Data trained on ResNet18")
      
ax.set_ylim(0, 1.2)
ax.legend(title="Detectors", bbox_to_anchor=(1.0, 1), loc='upper left')
ax.set_xticklabels(raw_data["plan_type"])
plt.xlabel("attacks")
plt.ylabel("n_95_frac")

plt.savefig(f"plots/{mode}_{model}_95_frac.png", bbox_inches='tight', dpi=300)